In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.utils import plot_model

2024-01-28 08:56:48.421925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Loading our trained autoencoder and encoder

In [14]:
import keras
loaded_encoder = keras.models.load_model("encoderJust.h5")
loaded_ae = keras.models.load_model("completeAE.h5")

# Loading our DATA

In [13]:
train_dir = '../DATASET/Normal_training/'

# Define batch size
batch_size = 64
SIZE = 128

# Function to load and preprocess images from a directory
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        try:
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=(SIZE, SIZE))
            img_array = img_to_array(img) / 255.0  # Rescale to [0, 1]
            images.append(img_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images)

# Load and preprocess training, validation, and anomaly images
train_generator = load_images_from_directory(train_dir)

Error loading image .DS_Store: cannot identify image file <_io.BytesIO object at 0x7fdfce2b38d0>
Error loading image IMG_0547.HEIC: cannot identify image file <_io.BytesIO object at 0x7fdfce2b38d0>


In [20]:
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
encoded_images = loaded_encoder.predict_generator(train_generator)

# Flatten the encoder output because KDE from sklearn takes 1D vectors as input
encoder_output_shape = loaded_encoder.output_shape #Here, we have 16x16x16
out_vector_shape = encoder_output_shape[1]*encoder_output_shape[2]*encoder_output_shape[3]

encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in encoded_images]

#Fit KDE to the image latent data
kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

/var/folders/f7/b7cb937d13v1w87kr83s6x080000gp/T/ipykernel_9184/2201707754.py:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  encoded_images = loaded_encoder.predict_generator(train_generator)


# VIDEO

In [31]:
#Now, input unknown images and sort as Good or Anomaly
def check_anomaly(encoder,img,folder, frame_count):
    density_threshold = 700 #Set this value based on (the observation)
    reconstruction_error_threshold = 0.1 # Set this value based on the above exercise
    img  = Image.open(img)
    saved_image = img
    img = np.array(img.resize((128,128), Image.ANTIALIAS))
    img = img / 255.
    img = img[np.newaxis, :,:,:]
    encoded_img = encoder.predict([[img]])
    encoded_img = [np.reshape(img, (out_vector_shape)) for img in encoded_img] 
    density = kde.score_samples(encoded_img)[0] 

    reconstruction = loaded_ae.predict([[img]])
    reconstruction_error = loaded_ae.evaluate([reconstruction],[[img]], batch_size = 1)[0]

    if density < density_threshold or reconstruction_error > reconstruction_error_threshold:
        saved_image.save(os.path.join(folder, f"frame_{frame_count:04d}.jpg"))

## After extracting the video's frames and saved them on 'frames' folder, lets loop through the folder and check anomaly for each frames, saving anomalies into 'anomalies' folder

In [32]:
#Load a couple of test images and verify whether they are reported as anomalies.
import glob
anomaly_file_path = glob.glob('Frames/*')
output_folder = "Anomalies"

for img in range(len(anomaly_file_path)):
    check_anomaly(loaded_encoder, anomaly_file_path[img], output_folder, img)

1/1 [==============================] - 0s 44ms/step


/var/folders/f7/b7cb937d13v1w87kr83s6x080000gp/T/ipykernel_9184/1425491494.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = np.array(img.resize((128,128), Image.ANTIALIAS))


1/1 [==============================] - 0s 43ms/step


/var/folders/f7/b7cb937d13v1w87kr83s6x080000gp/T/ipykernel_9184/1425491494.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = np.array(img.resize((128,128), Image.ANTIALIAS))


1/1 [==============================] - 0s 39ms/step


...

1/1 [==============================] - 0s 66ms/step


/var/folders/f7/b7cb937d13v1w87kr83s6x080000gp/T/ipykernel_9184/1425491494.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = np.array(img.resize((128,128), Image.ANTIALIAS))


1/1 [==============================] - 0s 58ms/step


/var/folders/f7/b7cb937d13v1w87kr83s6x080000gp/T/ipykernel_9184/1425491494.py:7: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = np.array(img.resize((128,128), Image.ANTIALIAS))


1/1 [==============================] - 0s 78ms/step - loss: 0.0805 - mse: 0.0805
